In [ ]:
import requests
import json
import urlparse
import logging
import pandas as pd
import sys
import datetime
from IPython.display import display

In [ ]:
class TLMtkData:
    def __init__(self,token):
        self.token=token
        self.root_url="https://api.wmcloud.com:443/data/v1/"
        self._initfuncmap()
    
    def _initfuncmap(self):
        self.funcmap={"getMktEqud":"api/market/getMktEqud.json"}
        
    def _getHttp(self,url,payload):
        head_auth = {"Authorization": "Bearer "+self.token}
        r=requests.get(url,params=payload,headers=head_auth)
        return r
        #payload = {'key1': 'value1', 'key2': 'value2'}
    
    def getMktEqud(self,secID="",ticker="",tradeDate="",field=""):
        """request
        secID	string	3选1	证券内部编码，可通过交易代码在getSecID获取到。（可多值输入，最多输入50只）
        ticker	string	3选1	股票交易代码，如'000001'（可多值输入，最多输入50只）
        tradeDate	Date	3选1	输入一个日期，不输入其他请求参数，可获取到一天全部沪深股票日行情数据，输入格式“YYYYMMDD”
        beginDate	Date	否	起始日期，输入格式“YYYYMMDD”
        endDate	Date	否	截止日期，输入格式“YYYYMMDD”
        isOpen	Int32	否	股票今日是否开盘标记位：0-今日未开盘，1-今日有开盘
        field	string	是	可选参数，用","分隔,默认为空，返回全部字段，不选即为默认值。返回字段见下方
        """
        if (secID=="" and ticker=="" and tradeDate==""):
            raise ValueError("secID,ticker,tradeDate 至少填写一个")
        
        req_params={"secID":secID,"ticker":ticker,"tradeDate":tradeDate,"field":field}
        
        desturl=urlparse.urljoin(self.root_url,self.funcmap["getMktEqud"])
        
        #print u"请求地址:",desturl
        
        r=self._getHttp(desturl,req_params)
        
        if (r.status_code!=200):
            logging.error("req "+r.url+r.status_code)
            return None
        
        
        jdata= r.json()
        if (jdata.get("data")==None):    
            if (jdata.get("retCode")=="-1"):
                logging.info(json.dumps(jdata))
            else:
                logging.error(json.dumps(jdata))      
            return None      
        return jdata
        """response
        secID	String	证券内部编码
        tradeDate	Date	交易日
        ticker	string	证券代码
        secShortName	string	证券简称
        exchangeCD	string	交易所代码
        preClosePrice	Double	昨收盘
        actPreClosePrice	Double	实际昨收盘
        openPrice	Double	今开盘
        highestPrice	Double	最高价
        lowestPrice	Double	最低价
        closePrice	Double	今收盘
        turnoverVol	Double	成交量
        turnoverValue	Double	成交金额
        dealAmount	Int32	成交笔数
        turnoverRate	Double	日换手率
        accumAdjFactor	Double	累积前复权因子，前复权是对历史行情进行调整，除权除息日的行情不受本次除权除息影响，不需进行调整。最新一次除权除息日至最新行情期间的价格不需要进行任何的调整，该期间前复权因子都是1。
        negMarketValue	Double	流通市值
        marketValue	Double	总市值
        PE	Double	滚动市盈率,将于2016-01-01停止维护，推荐使用/api/market/getStockFactorsOneDay或/api/market/getStockFactorsDateRange，返回字段PE
        PE1	Double	市盈率，将于2016-01-01停止维护，推荐使用/api/market/getStockFactorsOneDay或/api/market/getStockFactorsDateRange
        PB	Double	市净率，将于2016-01-01停止维护，推荐使用/api/market/getStockFactorsOneDay或/api/market/getStockFactorsDateRange，返回字段PB
        isOpen	Int32	是否开盘
        """  

In [ ]:
if __name__=="__main__":
    log_format = "%(filename)s [%(asctime)s] [%(levelname)s] %(message)s"
    logging.basicConfig(format=log_format,datefmt='%Y-%m-%d %H:%M:%S %p',level=logging.DEBUG)
    
    # 定义一个Handler打印Debug及以上级别的日志到sys.stderr  
    console = logging.StreamHandler()  
    console.setLevel(logging.DEBUG)  
    # 设置日志打印格式  
    formatter = logging.Formatter('%(name)-12s: %(levelname)-8s %(message)s') 
    console.setFormatter(formatter)  
    # 将定义好的console日志handler添加到root logger  
    logging.getLogger("").addHandler(console)  
    
    tk=TLMtkData(sys.argv[1])
    
    
    begin = datetime.date(2007, 1, 1)  
    end = datetime.date(2016,1,10)
    for i in range((end - begin).days+1):  
        day = begin + datetime.timedelta(days=i)
        daystr=datetime.datetime.strftime(day,'%Y%m%d')
        rsp=tk.getMktEqud(tradeDate=daystr)
        if (rsp==None):
            print daystr," error"
            continue
        ds=pd.DataFrame(rsp["data"])  
        ds.to_pickle("data/mtk_"+daystr+".pk")
        print daystr," done"
        
    